We used 2 different versions of scraping code for this project. The first one we used the one-by-one method to scrape data from article links; hence, this method took such long time to gain data. We used another method which is called 'multithreaded scraping' with max_worker = 5 to scrape data faster. Indeed, the data was scraped a lot faster (took half the time of the initial method); however, the data was missing a lot of features and the number of eliminated records were more than other categories.

### 0. Library

In [1]:
# Danh sách thư viện
import os
import csv
import json
import time
import requests
import concurrent.futures
import re 
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Hàm khởi tạo Selenium WebDriver
def create_chrome_driver():
    options = Options()
    options.add_argument("--headless=new") 
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver

### 1. Crawling links

In [2]:
def collect_links_vnexpress_bs(max_pages=12, category="giai-tri"): # Chuyên mục mặc định là giai-tri
    base_url = f"https://vnexpress.net/{category}" 
    article_pattern = re.compile(r"https:\/\/vnexpress\.net\/[a-zA-Z0-9\-]+(\.html)?$")
    all_links = set()

    for page in range(1, max_pages + 1):
        url = base_url if page == 1 else f"{base_url}-p{page}"

        print(f"\n🔍 Đang thu thập URL từ trang: {url}")
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status() 

            soup = BeautifulSoup(response.content, "html.parser")
            articles = soup.select('article a')

            page_links = set()
            for a_tag in articles:
                href = a_tag.get("href", "")
                if article_pattern.match(href):
                    page_links.add(href)

            all_links.update(page_links)
            print(f"✅ Đã thu thập {len(page_links)} link mới. Tổng: {len(all_links)}")

            if len(page_links) == 0:
                print("🚨 Không có link mới, dừng thu thập.")
                break

            time.sleep(1)  # Tránh bị chặn IP

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Lỗi khi truy cập {url}: {e}")
            continue

    return all_links

In [3]:
category = "thoi-su" # Chọn category muốn crawl link
links = collect_links_vnexpress_bs(max_pages=25, category=category)

file_name = f"{category}_article_links.txt"
file_folder = f"E:/UIT/Năm-2/Kì-2/Do-an-DS/Data/{category}"  # Đường dẫn workspace trên máy local

# Tạo thư mục nếu chưa có
os.makedirs(file_folder, exist_ok=True)

# Tạo đường dẫn đầy đủ cho file
file_path = os.path.join(file_folder, file_name)

# Ghi các liên kết vào file
with open(file_path, "w", encoding="utf-8") as f:
    for link in links:
        f.write(link + "\n")

print(f"📁 Đã lưu {len(links)} liên kết vào file: {file_path}")


🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su
✅ Đã thu thập 55 link mới. Tổng: 55

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p2
✅ Đã thu thập 30 link mới. Tổng: 82

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p3
✅ Đã thu thập 30 link mới. Tổng: 112

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p4
✅ Đã thu thập 30 link mới. Tổng: 142

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p5
✅ Đã thu thập 30 link mới. Tổng: 172

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p6
✅ Đã thu thập 30 link mới. Tổng: 199

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p7
✅ Đã thu thập 30 link mới. Tổng: 229

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p8
✅ Đã thu thập 30 link mới. Tổng: 256

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p9
✅ Đã thu thập 30 link mới. Tổng: 286

🔍 Đang thu thập URL từ trang: https://vnexpress.net/thoi-su-p10
✅ Đã thu thập 30 link mới. Tổng

### 2. Scraping article data

In [4]:
def process_single_article(link):
    driver = None
    try:
        driver = create_chrome_driver()
        driver.set_page_load_timeout(60)
        driver.set_script_timeout(60)

        try:
            driver.get(link)
            time.sleep(1)
        except TimeoutException:
            print(f"⚠️ Timeout khi load trang: {link}")
            driver.execute_script("window.stop();")

        # Lấy HTML của trang để xử lý với BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Tiêu đề (Sử dụng BeautifulSoup)
        try:
            title = soup.find("h1").text.strip()
        except Exception as e:
            title = "Không rõ"

        # Số từ (Sử dụng BeautifulSoup)
        try:
            wordcount_meta = soup.find("meta", attrs={"name": "its_wordcount"})
            word_count = int(wordcount_meta["content"]) if wordcount_meta and wordcount_meta.get("content", "").isdigit() else 0
        except Exception as e:
            word_count = 0

        # Ngày đăng (Sử dụng BeautifulSoup)
        try:
            publish_date = soup.find(class_="date").text.strip()
        except Exception as e:
            publish_date = "Không rõ"

        # Tags (Sử dụng BeautifulSoup)
        try:
            tags_elements = soup.select("div.tags a")
            tags = ', '.join([tag.text.strip() for tag in tags_elements if tag.text.strip()])
        except Exception as e:
            tags = ""

        # Số ảnh (Sử dụng BeautifulSoup)
        try:
            images = soup.select("img[itemprop='contentUrl']")
            image_count = len(images)
        except Exception as e:
            image_count = 0

        # Số video (Sử dụng BeautifulSoup)
        try:
            video_divs = soup.find_all("div", class_="box_embed_video")
            video_count = len(video_divs)
        except Exception as e:
            video_count = 0

        # Dữ liệu động: Sử dụng Selenium cho bình luận và tương tác
        # Số bình luận (Sử dụng Selenium)
        try:
            comment_text = driver.find_element(By.CLASS_NAME, 'section-comment').text.strip()
            match = re.search(r"(\d+)", comment_text)
            comments = match.group(1) if match else "0"
        except:
            comments = "0"

        # Tổng tương tác ở bình luận (Sử dụng Selenium)
        try:
            interaction_elements = driver.find_elements(By.CSS_SELECTOR, 'a.number')
            total_interactions = sum(int(i.text) for i in interaction_elements if i.text.isdigit())
        except:
            total_interactions = 0

        return [title, publish_date, word_count, comments, total_interactions, image_count, video_count, tags]

    except Exception as e:
        print(f"❌ Lỗi khi xử lý bài viết {link}:\n👉 {e}")
        return None

    finally:
        if driver:
            driver.quit()

In [5]:
# Hàm tạo tên file duy nhất
def get_unique_filename(folder, base_name, extension):
    count = 1
    filename = f"{base_name}{extension}"
    while os.path.exists(os.path.join(folder, filename)):
        filename = f"{base_name}_{count}{extension}"
        count += 1
    return filename

In [6]:
from concurrent.futures import ThreadPoolExecutor

def scrape_and_save_article_thread(link, chuyenmuc, folder_path, base_name, results, idx, total_links):
    result = process_single_article(link)  # Hàm scrape dữ liệu của một bài viết
    if result:
        results.append({
            "Tiêu đề": result[0],
            "Ngày đăng": result[1],
            "Số từ": result[2],
            "Số bình luận": result[3],
            "Tổng tương tác ở bình luận": result[4],
            "Số ảnh": result[5],
            "Số video": result[6],
            "Tags": result[7],
            "Chuyên mục": chuyenmuc
        })
        print(f"✅ Bài viết {idx + 1}/{total_links} đã thành công: {link}")
    else:
        print(f"⚠️ Bỏ qua bài viết {idx + 1}/{total_links} do lỗi: {link}")

# Hàm scrape và lưu dữ liệu bằng cách sử dụng multi-threading
def scrape_and_save_articles_multithreaded(all_links, folder_path, base_name, chuyenmuc, max_workers=4):
    # Tạo đường dẫn cho file .csv và .json
    csv_file_name = get_unique_filename(folder_path, base_name, ".csv")
    json_file_name = get_unique_filename(folder_path, base_name, ".json")

    csv_path = os.path.join(folder_path, csv_file_name)
    json_path = os.path.join(folder_path, json_file_name)

    os.makedirs(folder_path, exist_ok=True)
    
    results = []  # Danh sách để lưu dữ liệu bài viết
    total_links = len(all_links)  # Tổng số link cần xử lý

    # Chạy multi-threading với ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for idx, link in enumerate(all_links):
            futures.append(executor.submit(scrape_and_save_article_thread, link, chuyenmuc, folder_path, base_name, results, idx, total_links))
        
        # Chờ tất cả các thread hoàn thành
        for future in futures:
            future.result()  # Đảm bảo là tất cả threads hoàn thành

    # Ghi kết quả vào file CSV và JSON sau khi thu thập hết dữ liệu
    if len(results) > 0:
        with open(csv_path, mode='w', encoding='utf-8-sig', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["Tiêu đề", "Ngày đăng", "Số từ", "Số bình luận", "Tổng tương tác ở bình luận", "Số ảnh", "Số video", "Tags", "Chuyên mục"])
            for result in results:
                writer.writerow([result["Tiêu đề"], result["Ngày đăng"], result["Số từ"], result["Số bình luận"],
                                 result["Tổng tương tác ở bình luận"], result["Số ảnh"], result["Số video"], result["Tags"], result["Chuyên mục"]])

        # Lưu kết quả vào file .json
        with open(json_path, 'w', encoding='utf-8') as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)

        print(f"📁 Đã lưu {len(results)} bài viết vào file: {csv_file_name} và {json_file_name}")
    else:
        print(f"❌ Không có bài viết nào được lưu. Các file đã bị xóa.")
        os.remove(csv_path)
        os.remove(json_path)

In [7]:
category = "thoi-su"
file_folder = r"E:\UIT\Năm-2\Kì-2\Do-an-DS\Data\thoi-su"
base_name = f"{category}_articles_data"

# Chia danh sách link và chạy đa luồng
link_lists = list(links)
scrape_and_save_articles_multithreaded(link_lists[:], file_folder, base_name, category)

✅ Bài viết 4/613 đã thành công: https://vnexpress.net/chu-tich-nuoc-dang-huong-gio-to-hung-vuong-4870850.html
✅ Bài viết 1/613 đã thành công: https://vnexpress.net/nguoi-dong-bhxh-tu-nguyen-du-kien-huong-luong-huu-the-nao-tu-1-7-4868528.html
✅ Bài viết 5/613 đã thành công: https://vnexpress.net/nguoi-noi-tieng-quang-cao-sai-su-that-co-the-bi-han-che-xuat-hien-4870732.html
✅ Bài viết 2/613 đã thành công: https://vnexpress.net/be-trai-ngu-quen-tren-cay-hon-100-nguoi-suot-dem-tim-kiem-4869925.html
✅ Bài viết 8/613 đã thành công: https://vnexpress.net/du-an-tai-dinh-cu-gan-50-ty-dong-bo-hoang-4863397.html
✅ Bài viết 3/613 đã thành công: https://vnexpress.net/khong-to-chuc-thanh-tra-bo-va-huyen-4867205.html
✅ Bài viết 6/613 đã thành công: https://vnexpress.net/cam-xe-5-duong-trung-tam-tp-hcm-de-lap-dat-tran-dia-phao-4870041.html
✅ Bài viết 7/613 đã thành công: https://vnexpress.net/can-chinh-sach-giup-can-bo-tinh-gian-chuyen-sang-khu-vuc-tu-4865805.html
✅ Bài viết 10/613 đã thành công: http